### 1. Preparation with Classic Method

In [1]:
from qiskit.primitives import StatevectorEstimator as Estimator
from qc4pyscf.ansatz.ADAPT_VQE import ADAPT_VQE
from qc4pyscf.ansatz.UCC import UCC
from pyscf import gto,scf,fci

###########################################
# Original PySCF Process: 
# 1. Make mol object
# 2. Make RHF or UHF object
# 3. Run it
###########################################

# 1. Make mol object
mol     = gto.M(atom = "H 0.000000 0.000000 0; H 0.000000 0.000000 0.720000", 
                basis = "sto3g",
                charge = 0,
                spin = 0,
                verbose = 0)

# 2. Make HF object and run
mf      = scf.RHF(mol)
E_hf    = mf.kernel()

# Code for reference
mf_fci  = fci.FCI(mf)
E_fci   = mf_fci.kernel()[0]

print(f" HF: {E_hf:.3f} Hartree\nFCI: {E_fci:.3f} Hartree\n\nCorrelation Energy: {E_fci - E_hf:.3f} Hartree")

 HF: -1.117 Hartree
FCI: -1.137 Hartree

Correlation Energy: -0.020 Hartree


### 2-1. Calculate with Quantum Algorithm - UCC

In [2]:
###########################################
# Additional Process for Quantum Computer:
# 1. Put variables in UCC object as below
# 2. Build quantum ingredients
# 3. Run it
###########################################

# 0. Make a proper cost function.
def cost_func(params, ansatz, H, estimator):
    res = estimator.run([(ansatz, H, params)]).result()
    energy = res[0].data.evs
    return energy

# 1. Make UCC object. Settings are same as Default
qc_mf = UCC(mol           = mol,                # gto.Mole Object
            mf            = mf,                 # PySCF's scf or dft Object
            ex_code       = 'sd',               # Excitation Code: (sd)
            mapping       = 'jordan_wigner',    # Mapping: (jordan_wignar)
            cd_acc        = 1e-6,               # Cholesty Decomposition Accuracy
            max_iteration = 200000,             # Minimizer Maximum Iteration
            amplitudes    = [],                 # Amplitudes of Excitation Evolvers
            spin_symm     = True)               # Excitation Spin Symmetry (It changes into False if the system is open shell)
            
# 2. Build QC Ingredients
qc_mf.build()

# 3. Run UCC Algorithm
qc_mf.run(  cost_func,                      # Cost function to use
            Estimator(),                    # Quantum estimator
            minimize_algorithm='COBYLA',    # Mimization Algorithm for scipy minimize
            on_ansatz_ftn=None)             # Function for ansatz between building and operating e.g. pm.run()


accuracy of Cholesky decomposition  7.771561172376096e-16
Alpha - Alpha Excitation Denied
Beta - Beta Excitation Denied
Build Done
     Minimize Algorithm     : COBYLA
  Max Minimizing Iteration  : 200000
      Excitation Code       : SD
     Mapping Algorithm      : Jordan-Wigner
        C.D. Accuracy       : 1.0e-06
     Initial Conditions     : False
Calculated Energy: -1.1371117026192732 Hartree | Minimize Success: True | Computation Time: 0.264s


np.float64(-1.1371117026192732)

### 2-2. Calculate with Quantum Algorithm - ADAPT-VQE

In [3]:
###########################################
# Additional Process for Quantum Computer:
# 1. Put variables in ADAPT-VQE object as below
# 2. Build quantum ingredients
# 3. Run it
###########################################

# 0. Make a proper cost function.
def cost_func(params, ansatz, H, estimator):
    res = estimator.run([(ansatz, H, params)]).result()
    energy = res[0].data.evs
    return energy

# 1. Make ADAPT-VQE object. Settings are same as Default
qc_mf = ADAPT_VQE(  mol           = mol,                # gto.Mole Object
                    mf            = mf,                 # PySCF's scf or dft Object
                    ex_code       = 'sd',               # Excitation Code: (sd)
                    mapping       = 'jordan_wigner',    # Mapping: (jordan_wignar)
                    cd_acc        = 1e-6,               # Cholesky Decomposition Accuracy
                    energy_conv   = 1e-6,               # Energy Converge Tolerance
                    grad_conv     = 1e-2,               # 2-Norm of Gradient Vector Converge Tolerance
                    conv_type     = 0,                  # 0: Grad | 1: Energy | 2: Grad and Energy
                    max_cycle     = 50,                 # ADAPT-VQE Maximum Cycle
                    max_iteration = 200000,             # Minimizer Maximum Iteration
                    initial       = None,               # Initial Ansatz JSON Directory
                    chkfile       = None,               # Chkfile Directory
                    spin_symm     = True)               # Spin Symmetry
            
# 2. Build QC Ingredients
qc_mf.build()

# 3. Run UCC Algorithm
qc_mf.run(  cost_func,                      # Cost function to use
            Estimator(),                    # Quantum estimator
            minimize_algorithm='BFGS',      # Mimization Algorithm for scipy minimize
            on_ansatz_ftn=None)             # Function for ansatz between building and operating e.g. pm.run()

accuracy of Cholesky decomposition  7.771561172376096e-16
Build Done. #Operator in Operator Pool: 36
     Minimize Algorithm     : BFGS
  Max Minimizing Iteration  : 200000
     Max ADAPT-VQE Cycle    : 50
      Excitation Code       : SD
     Mapping Algorithm      : Jordan-Wigner
        C.D. Accuracy       : 1.00e-06
     Converge Tolerance     : E(1.00e-06) | Grad(1.00e-02)
      Convergence Type      : 0 (0: Grad | 1: Energy | 2: Grad and Energy)
         Checkfile          : None
#Evolver:     0 | Grad_Norm: 1.01879 | Used Op.:  a1/b1//b0/a0 (+) b1/a1//a0/b0   | Minimize: Success | Energy: -1.137111715 AU | dE: -2.137111715 AU | Computation Time for Cycle: 0.092s
#Evolver:     1 | Grad_Norm: 0.00000 | Used Op.:  a1/b1//b0/a0 (+) b1/a1//a0/b0   | Minimize: Success | Energy: -1.137111715 AU | dE: 0.000000000 AU | Computation Time for Cycle: 0.197s
System Converged. E = -1.1371117151154442 Hartree | Total Computation Time: 0.289s


-1.1371117151154442